#Name: Ramya Banda
#ECE 595 Machine Learning II
#Project 5: NTM Part 1 - Code

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)
#%cd gdrive/My Drive/Neural_Turing_Machine/NTM_small
%cd gdrive/My Drive/NTM_Student

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
/content/gdrive/My Drive/NTM_Student


In [ ]:
from utils import OmniglotDataLoader, one_hot_decode, five_hot_decode
import tensorflow as tf
import argparse
import numpy as np
%tensorflow_version 1.x
print(tf.__version__)


1.15.0


In [ ]:
class NTMOneShotLearningModel():
  def __init__(self, model, n_classes, batch_size, seq_length, image_width, image_height,
                rnn_size, num_memory_slots, rnn_num_layers, read_head_num, write_head_num, memory_vector_dim, learning_rate):
    self.output_dim = n_classes

    # Note: the images are flattened to 1D tensors
    # The input data structure is of the following form:
    # self.x_image[i,j,:] = jth image in the ith sequence (or, episode)
    self.x_image = tf.placeholder(dtype=tf.float32, shape=[batch_size, seq_length, image_width * image_height])
    # Model's output label is one-hot encoded
    # The data structure is of the following form:
    # self.x_label[i,j,:] = one-hot label of the jth image in 
    #             the ith sequence (or, episode)
    self.x_label = tf.placeholder(dtype=tf.float32, shape=[batch_size, seq_length, self.output_dim])
    # Target label is one-hot encoded
    self.y = tf.placeholder(dtype=tf.float32, shape=[batch_size, seq_length, self.output_dim])
    
    # The dense layer for mapping controller output and retrieved
    # memory content to classification labels
    self.controller_output_to_ntm_output = tf.keras.layers.Dense(units=self.output_dim, use_bias=True)

    if model == 'LSTM':
      # Using a LSTM layer to serve as the controller, no memory
      def rnn_cell(rnn_size):
        return tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
      cell = tf.nn.rnn_cell.MultiRNNCell([rnn_cell(rnn_size) for _ in range(rnn_num_layers)])
      state = cell.zero_state(batch_size=batch_size, dtype=tf.float32)
    
    # Initialize the controller model, including wiping its memory
    # Also, get the initial state of the MANN model
    
    self.state_list = [state]
    # Setup the NTM's output
    self.o = []
    
    # Now iterate over every sample in the sequence 
    for t in range(seq_length):
      output, state = cell(tf.concat([self.x_image[:, t, :], self.x_label[:, t, :]], axis=1), state)
      # Map controller output (with retrieved memory) + current (offseted) label 
      # to the overall ntm's output with an affine operation
      # The output is the classification labels
      output = self.controller_output_to_ntm_output(output)
      output = tf.nn.softmax(output, axis=1)
      self.o.append(output)
      self.state_list.append(state)
    # post-process the output of the classifier
    self.o = tf.stack(self.o, axis=1)
    self.state_list.append(state)

    eps = 1e-8
    # cross entropy, between model output labels and target labels
    self.learning_loss = -tf.reduce_mean(  
        tf.reduce_sum(self.y * tf.log(self.o + eps), axis=[1, 2])
    )
    
    self.o = tf.reshape(self.o, shape=[batch_size, seq_length, -1])
    self.learning_loss_summary = tf.summary.scalar('learning_loss', self.learning_loss)

    with tf.variable_scope('optimizer'):
      self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
      self.train_op = self.optimizer.minimize(self.learning_loss)

The training and testing functions

In [ ]:
def train(learning_rate, image_width, image_height, n_train_classes, n_test_classes, restore_training, \
         num_epochs, n_classes, batch_size, seq_length, num_memory_slots, augment, save_dir, model_path, tensorboard_dir):
  
  # We always use one-hot encoding of the labels in this experiment
  label_type = "one_hot"

  # Initialize the model
  model = NTMOneShotLearningModel(model=model_path, n_classes=n_classes,\
                    batch_size=batch_size, seq_length=seq_length,\
                    image_width=image_width, image_height=image_height, \
                    rnn_size=rnn_size, num_memory_slots=num_memory_slots,\
                    rnn_num_layers=rnn_num_layers, read_head_num=read_head_num,\
                    write_head_num=write_head_num, memory_vector_dim=memory_vector_dim,\
                    learning_rate=learning_rate)
  print("Model initialized")
  data_loader = OmniglotDataLoader(
      image_size=(image_width, image_height),
      n_train_classses=n_train_classes,
      n_test_classes=n_test_classes
  )
  print("Data loaded")
  with tf.Session() as sess:
    if restore_training:
      saver = tf.train.Saver()
      ckpt = tf.train.get_checkpoint_state(save_dir + '/' + model_path)
      saver.restore(sess, ckpt.model_checkpoint_path)
    else:
      saver = tf.train.Saver(tf.global_variables())
      tf.global_variables_initializer().run()
    train_writer = tf.summary.FileWriter(tensorboard_dir + '/' + model_path, sess.graph)
    print("1st\t2nd\t3rd\t4th\t5th\t6th\t7th\t8th\t9th\t10th\tepoch\tloss")
    for b in range(num_epochs):
      # Test the model
      if b % 100 == 0:
        # Note: the images are flattened to 1D tensors
        # The input data structure is of the following form:
        # x_image[i,j,:] = jth image in the ith sequence (or, episode)
        # And the sequence of 50 images x_image[i,:,:] constitute
        # one episode, and each class (out of 5 classes) has around 10
        # appearances in this sequence, as seq_length = 50 and 
        # n_classes = 5, as specified in the code block below
        # See the details in utils.py, OmniglotDataLoader class
        x_image, x_label, y = data_loader.fetch_batch(n_classes, batch_size, seq_length,
                                  type='test',
                                  augment=augment,
                                  label_type=label_type)
        feed_dict = {model.x_image: x_image, model.x_label: x_label, model.y: y}
        output, learning_loss = sess.run([model.o, model.learning_loss], feed_dict=feed_dict)
        merged_summary = sess.run(model.learning_loss_summary, feed_dict=feed_dict)
        train_writer.add_summary(merged_summary, b)
        accuracy = test(seq_length, y, output)
        for accu in accuracy:
          print('%.4f' % accu, end='\t')
        print('%d\t%.4f' % (b, learning_loss))

      # Save model per 2000 epochs
      if b%2000==0 and b>0:
        saver.save(sess, save_dir + '/' + model_path + '/model.tfmodel', global_step=b)

      # Train the model
      x_image, x_label, y = data_loader.fetch_batch(n_classes, batch_size, seq_length, \
                                type='train',
                                augment=augment,
                                label_type=label_type)
      feed_dict = {model.x_image: x_image, model.x_label: x_label, model.y: y}
      sess.run(model.train_op, feed_dict=feed_dict)
      
def test(seq_length, y, output):
  # Fill in
  total_acc = np.zeros(10)
  total_class = np.zeros(10)
  for l in range(output.shape[0]):
    class_tracker = np.zeros(5)
    acc = np.zeros(50)
    for i in range(seq_length):
      class_tracker[np.argmax(output[l][i])] = class_tracker[np.argmax(output[l][i])] + 1
      j = int(round(class_tracker[np.argmax(output[l][i])]))
      if np.argmax(output[l][i]) == np.argmax(y[l][i]):
        acc[j] = acc[j] + 1
    for j in range(10):
      total_acc[j] = total_acc[j] + acc[j]
      total_class[j] = total_class[j] + len(class_tracker[class_tracker >= j])
  return ((total_acc/total_class))

In [ ]:
restore_training = False
label_type = "one_hot"
n_classes = 5
seq_length = 50
augment = True
read_head_num = 4
batch_size = 16
num_epochs = 100000
learning_rate = 1e-3
rnn_size = 200
image_width = 20
image_height = 20
rnn_num_layers = 1
num_memory_slots = 128
memory_vector_dim = 40
shift_range = 1
write_head_num = 4
test_batch_num = 100
n_train_classes = 220
n_test_classes = 60
save_dir = './save/one_shot_learning'
tensorboard_dir = './summary/one_shot_learning'
model_path = 'LSTM'
train(learning_rate, image_width, image_height, n_train_classes, n_test_classes, restore_training, \
         num_epochs, n_classes, batch_size, seq_length, num_memory_slots, augment, save_dir, model_path, tensorboard_dir)


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model initialized
Entered Dataloader
10.0% data loaded.
20.0% data loaded.
30.0% data loaded.
40.0% data loaded.
50.0% data loaded.
60.0% data loaded.
70.0% data loaded.
80.0% data loaded.
90.0% data loaded.
100.0% data loaded.
Data loaded
1st	2nd	3rd	4th	5th	6th	7th	8th	9th	10th	epoch	loss
0.0000	0.1558	0.1944	0.1940	0.2381	0.2069	0.1538	0.2041	0.1905	0.1212	0	80.9050
0.0000	0.1667	0.2188	0.1800	0.1463	0.2432	0.1622	0.2162	0.3714	0.2059	100	80.5165
0.0000	0.

Exception ignored in: <bound method Image.__del__ of <PIL.Image.Image image mode=L size=105x105 at 0x7FE3B2758EB8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 585, in __del__
    if (hasattr(self, 'fp') and hasattr(self, '_exclusive_fp')
KeyboardInterrupt


0.0000	0.4250	0.5000	0.6000	0.6835	0.6795	0.6447	0.6970	0.8136	0.6800	14700	44.7525
0.0000	0.3500	0.5625	0.6125	0.6962	0.6494	0.6892	0.6812	0.7812	0.7193	14800	41.1734
0.0000	0.4375	0.5375	0.5625	0.5823	0.6486	0.7059	0.6562	0.7500	0.7347	14900	44.5131
0.0000	0.4750	0.5750	0.6125	0.6410	0.7143	0.7200	0.7164	0.7143	0.7843	15000	42.9138
0.0000	0.4250	0.5500	0.6625	0.7468	0.6351	0.6944	0.7391	0.8136	0.8222	15100	39.0681


KeyboardInterrupt: ignored